In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gffutils

In [ ]:
A_contigs_df = pd.read_csv('./CM3/final_results/abyss/A_contigs.tsv', sep='\t')
A_contigs_df['contig_length'] = A_contigs_df['contig'].apply(lambda x: len(x))
A_contigs_df.shape

In [ ]:

db_path = 'd:/scRNA/Thesis-Code-Base-master/Data/annotation/'
psl_file = './CM3/final_results/abyss/A_contigs_alignment.psl'
contigs_file ='./CM3/final_results/abyss/A_contigs.tsv'

In [ ]:
db = gffutils.FeatureDB(db_path + "GENCODE_v45.db")
polyA_db = gffutils.FeatureDB(db_path + "polyA_v45.db")
tRNA_db = gffutils.FeatureDB(db_path + "tRNA_v45.db")
psl_df = pd.read_csv(psl_file, delimiter='\t', comment='#', header=None, names=['name', 'subject', 'identity', 'length', 'mismatches', 'gap_openings', 'q_start', 'q_end', 's_start', 's_end', 'e_value', 'bit_score'])
contigs_df = pd.read_csv(contigs_file, sep='\t')

In [ ]:

contigs_df['contig_length'] = contigs_df['contig'].apply(lambda x: len(x))
contigs_df = contigs_df[contigs_df['contig_length'] >= 31]
psl_df = psl_df.merge(contigs_df, on='name', how='left')
psl_df = psl_df[psl_df['mismatches']/psl_df['contig_length']<=0.05]
psl_df = psl_df[psl_df['contig_length']==psl_df['length']]
psl_df.sort_values(by='e_value', inplace=True)
print('psl count: ', psl_df.shape[0] )
lncRNAs = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
pseudoGenes = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
retained_introns = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
transcribed_unprocessed_pseudogenes = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
miRNAs = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
polyAs = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
tRNAs = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
exons = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
unmapped = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
CDS = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}
UTR = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}

gene = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}

start_codon = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}

stop_codon = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}

transcript = {
    'name': [],
    'contig': [],
    'chr': [],
    'start': [],
    'end': [],
    'gene_name': [],
}

In [ ]:
def add_to_annotation_dict(annotation_dict, feature, name, contig, chr, start, end):
    annotation_dict['name'].append(name)
    annotation_dict['contig'].append(contig)
    annotation_dict['chr'].append(chr)
    annotation_dict['start'].append(start)
    annotation_dict['end'].append(end)
    annotation_dict['gene_name'].append(feature.attributes['gene_name'][0])


In [ ]:
cnt = 0
f_types = set()
for index, row in psl_df.iterrows():
    chr, start, end = row['subject'], row['s_start'], row['s_end']
    features = db.region(region=(chr, start, end))
    for f in features:
        f_types.add(f.featuretype)
        if f.attributes['gene_type'][0]=='lncRNA':
            add_to_annotation_dict(lncRNAs, f, row['name'], row['contig'], chr, start, end)
        if f.attributes['gene_type'][0]=='miRNA':
            add_to_annotation_dict(miRNAs, f, row['name'], row['contig'], chr, start, end)
        if 'transcript_type' in f.attributes:
            if f.attributes['transcript_type'][0]=='retained_intron':
                add_to_annotation_dict(retained_introns, f, row['name'], row['contig'], chr, start, end)
        if f.attributes['gene_type'][0]=='processed_pseudogene':
            add_to_annotation_dict(pseudoGenes, f, row['name'], row['contig'], chr, start, end)
        if f.attributes['gene_type'][0]=='transcribed_processed_pseudogene':
            add_to_annotation_dict(transcribed_unprocessed_pseudogenes, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='exon':
            add_to_annotation_dict(exons, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='CDS':
            add_to_annotation_dict(CDS, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='UTR':
            add_to_annotation_dict(UTR, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='gene':
            add_to_annotation_dict(gene, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='start_codon':
            add_to_annotation_dict(start_codon, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='stop_codon':
            add_to_annotation_dict(stop_codon, f, row['name'], row['contig'], chr, start, end)
        if f.featuretype=='transcript':
            add_to_annotation_dict(transcript, f, row['name'], row['contig'], chr, start, end)


In [ ]:
for index, row in psl_df.iterrows():
    chr, start, end = row['subject'], row['s_start'], row['s_end']
    features = polyA_db.region(region=(chr, start, end))
    for f in features:
        add_to_annotation_dict(polyAs, f, row['name'], row['contig'], chr, start, end)

In [ ]:
for index, row in psl_df.iterrows():
  chr, start, end = row['subject'], row['s_start'], row['s_end']
  features = tRNA_db.region(region=(chr, start, end))
  for f in features:
      add_to_annotation_dict(tRNAs, f, row['name'], row['contig'], chr, start, end)

In [ ]:

lncRNAs_df = pd.DataFrame(lncRNAs)
lncRNAs_df.to_csv(contigs_file.replace('.tsv', '_lncRNA.tsv'), sep='\t', index=False) 

retained_introns_df = pd.DataFrame(retained_introns)
retained_introns_df.to_csv(contigs_file.replace('.tsv', '_retained_introns.tsv'), sep='\t', index=False)

pseudoGenes_df = pd.DataFrame(pseudoGenes)
pseudoGenes_df.to_csv(contigs_file.replace('.tsv', '_pseudoGenes.tsv'), sep='\t', index=False)

transcribed_unprocessed_pseudogenes_df = pd.DataFrame(transcribed_unprocessed_pseudogenes)
transcribed_unprocessed_pseudogenes_df.to_csv(contigs_file.replace('.tsv', '_transcribed_unprocessed_pseudogenes.tsv'), sep='\t', index=False)

polyAs_df = pd.DataFrame(polyAs)
polyAs_df.to_csv(contigs_file.replace('.tsv', '_polyAs.tsv'), sep='\t', index=False)

tRNAs_df = pd.DataFrame(tRNAs)
tRNAs_df.to_csv(contigs_file.replace('.tsv', '_tRNAs.tsv'), sep='\t', index=False)

miRNAs_df = pd.DataFrame(miRNAs)
miRNAs_df.to_csv(contigs_file.replace('.tsv', '_miRNAs.tsv'), sep='\t', index=False)
exons_df = pd.DataFrame(exons)  

exons_df.to_csv(contigs_file.replace('.tsv', '_exons.tsv'), sep='\t', index=False)
CDS_df = pd.DataFrame(CDS)
CDS_df.to_csv(contigs_file.replace('.tsv', '_CDS.tsv'), sep='\t', index=False)

UTR_df = pd.DataFrame(UTR)
UTR_df.to_csv(contigs_file.replace('.tsv', '_UTR.tsv'), sep='\t', index=False)
start_codon_df = pd.DataFrame(start_codon)
start_codon_df.to_csv(contigs_file.replace('.tsv', '_start_codon.tsv'), sep='\t', index=False)
stop_codon_df = pd.DataFrame(stop_codon)
stop_codon_df.to_csv(contigs_file.replace('.tsv', '_stop_codon.tsv'), sep='\t', index=False)
gene_df = pd.DataFrame(gene)
gene_df.to_csv(contigs_file.replace('.tsv', '_gene.tsv'), sep='\t', index=False)
transcript_df = pd.DataFrame(transcript)
transcript_df.to_csv(contigs_file.replace('.tsv', '_transcript.tsv'), sep='\t', index=False)


In [ ]:
print('lncRNAs count: ', lncRNAs_df.shape[0])
print('retained_introns count: ', retained_introns_df.shape[0])
print('pseudoGenes count: ', pseudoGenes_df.shape[0])
print('transcribed_unprocessed_pseudogenes count: ', transcribed_unprocessed_pseudogenes_df.shape[0])
print('polyAs count: ', polyAs_df.shape[0])
print('tRNAs count: ', tRNAs_df.shape[0])
print('miRNAs count: ', miRNAs_df.shape[0])
print('exons count: ', exons_df.shape[0])
print('CDS count: ', CDS_df.shape[0])
print('UTR count: ', UTR_df.shape[0])
print('start_codon count: ', start_codon_df.shape[0])
print('stop_codon count: ', stop_codon_df.shape[0])
print('gene count: ', gene_df.shape[0])
print('transcript count: ', transcript_df.shape[0])

In [ ]:
all_mapped_contigs = pd.concat([lncRNAs_df, 
                                retained_introns_df, 
                                pseudoGenes_df, 
                                transcribed_unprocessed_pseudogenes_df, 
                                polyAs_df, 
                                tRNAs_df, 
                                miRNAs_df,
                                exons_df,
                                CDS_df,
                                UTR_df,
                                start_codon_df,
                                stop_codon_df,
                                gene_df,
                                transcript_df
                                ])


In [ ]:
unannotated_df = psl_df[~psl_df['name'].isin(all_mapped_contigs['name'])]

In [ ]:
unannotated_df.to_csv(contigs_file.replace('.tsv', '_unannotated.tsv'), sep='\t', index=False)

In [ ]:
unmapped_df = contigs_df[~contigs_df['name'].isin(psl_df['name'])]

In [ ]:
unmapped_df.to_csv(contigs_file.replace('.tsv', '_unmapped.tsv'), sep='\t', index=False)